In [ ]:
import json
import math
from collections import OrderedDict
import torch
from torch import nn, Tensor
from typing import Union, Tuple, List, Iterable, Dict
import torch.nn.functional as F
from torch.nn.parameter import Parameter
from torch.optim import AdamW
from torch.utils.data import DataLoader
from scipy.stats import pearsonr, spearmanr
import numpy as np
import gzip, csv
import pandas as pd
from tqdm.auto import tqdm

torch.manual_seed(0)
np.random.seed(0)

In [ ]:
! pip3 install datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.3 MB/s eta 0:00:00


In [ ]:
%pip install transformers
from transformers import AutoTokenizer
# If you can not find all the bugs, use the line below for AutoModel
#from transformers import AutoModel


In [ ]:
! pip3 install avalanche-lib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 971.9/971.9 kB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.4/532.4 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.4/840.4 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.5/258.5 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 960.0/960.0 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 737.2/737.2 kB 

In [ ]:
! pip3 install torch transformers

  Using cached avalanche-0.3.0.tar.gz (14 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
from collections import OrderedDict

def gelu(x):
    """Implementation of the gelu activation function."""
    return x * 0.5 * (1.0 + torch.erf(x / math.sqrt(2.0)))

class Config(object):
    """Configuration class to store the configuration of a `BertModel`."""
    def __init__(self,
                 vocab_size,
                 hidden_size=512,
                 num_hidden_layers=6,
                 num_attention_heads=8,
                 intermediate_size=2048,
                 dropout_prob=0.1,
                 max_position_embeddings=512,
                 type_vocab_size=2,
                 initializer_range=0.02):
        """Constructs Config for BertModel."""
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        self.num_hidden_layers = num_hidden_layers
        self.num_attention_heads = num_attention_heads
        self.intermediate_size = intermediate_size
        self.hidden_dropout_prob = dropout_prob
        self.attention_probs_dropout_prob = dropout_prob
        self.max_position_embeddings = max_position_embeddings
        self.type_vocab_size = type_vocab_size
        self.initializer_range = initializer_range

    @classmethod
    def from_dict(cls, dict_object):
        """Constructs Config from a Python dictionary of parameters."""
        config = Config(vocab_size=None)
        for (key, value) in dict_object.items():
            config.__dict__[key] = value
        return config

class LayerNorm(nn.Module):
    """Layer normalization module."""
    def __init__(self, hidden_size, variance_epsilon=1e-12):
        """Constructs LayerNorm object for Transformer layer in BERT model."""
        super(LayerNorm, self).__init__()
        self.gamma = nn.Parameter(torch.ones(hidden_size))
        self.beta = nn.Parameter(torch.zeros(hidden_size))
        self.variance_epsilon = variance_epsilon

    def forward(self, x):
        """Forward pass of the LayerNorm layer."""
        u = x.mean(-1, keepdim=True)
        s = (x - u).pow(2).mean(-1, keepdim=True)
        x = (x - u) / torch.sqrt(s + self.variance_epsilon)
        return self.gamma * x + self.beta

class MLP(nn.Module):
    """Feed forward network with gelu activation."""
    def __init__(self, hidden_size, intermediate_size):
        """Constructs MLP object for Transformer layer in BERT model."""
        super(MLP, self).__init__()
        self.dense_expansion = nn.Linear(hidden_size, intermediate_size)
        self.dense_contraction = nn.Linear(intermediate_size, hidden_size)

    def forward(self, x):
        """Forward pass of the MLP layer."""
        x = self.dense_expansion(x)
        x = self.dense_contraction(gelu(x))
        return x

class Layer(nn.Module):
    """The Transformer layer."""
    def __init__(self, config):
        """Constructs Layer object for Transformer layer in BERT model based on config."""
        super(Layer, self).__init__()

        self.hidden_size = config.hidden_size
        self.num_attention_heads = config.num_attention_heads
        self.attention_head_size = int(config.hidden_size / config.num_attention_heads)
        self.all_head_size = self.num_attention_heads * self.attention_head_size

        self.query = nn.Linear(config.hidden_size, self.all_head_size)
        self.key = nn.Linear(config.hidden_size, self.all_head_size)
        self.value = nn.Linear(config.hidden_size, self.all_head_size)

        self.dropout = nn.Dropout(config.attention_probs_dropout_prob)

        self.attn_out = nn.Linear(config.hidden_size, config.hidden_size)
        self.ln1 = LayerNorm(config.hidden_size)

        self.mlp = MLP(config.hidden_size, config.intermediate_size)
        self.ln2 = LayerNorm(config.hidden_size)

    def split_heads(self, tensor, num_heads, attention_head_size):
        """Split hidden_size into num_heads * attention_head_size and transpose into shape [batch, num_heads, seq_len, attention_head_size]."""
        new_shape = tensor.size()[:-1] + (num_heads, attention_head_size)
        tensor = tensor.view(*new_shape)
        return tensor.permute(0, 2, 1, 3).contiguous()

    def merge_heads(self, tensor, num_heads, attention_head_size):
        """Transpose and then reshape into shape [batch, seq_len, hidden_size]."""
        tensor = tensor.permute(0, 2, 1, 3).contiguous()
        new_shape = tensor.size()[:-2] + (num_heads * attention_head_size,)
        return tensor.view(new_shape)

    def attn(self, q, k, v, attention_mask):
        """Attention mechanism for the Transformer layer."""
        mask = attention_mask == 1
        mask = mask.unsqueeze(1).unsqueeze(2)

        s = torch.matmul(q, k.transpose(-1, -2))
        s = s / math.sqrt(self.attention_head_size)

        s = torch.where(mask, s, torch.tensor(float('-inf')))

        p = F.softmax(s, dim=-1)
        p = self.dropout(p)

        a = torch.matmul(p, v)
        return a

    def forward(self, x, attention_mask):
        """Forward pass of the Transformer layer in BERT."""
        q, k, v = self.query(x), self.key(x), self.value(x)

        q = self.split_heads(q, self.num_attention_heads, self.attention_head_size)
        k = self.split_heads(k, self.num_attention_heads, self.attention_head_size)
        v = self.split_heads(v, self.num_attention_heads, self.attention_head_size)

        a = self.attn(q, k, v, attention_mask)
        a = self.merge_heads(a, self.num_attention_heads, self.attention_head_size)
        a = self.attn_out(a)
        a = self.dropout(a)
        a = self.ln1(a + x)

        m = self.mlp(a)
        m = self.dropout(m)
        m = self.ln2(m + a)

        return m

class Bert(nn.Module):
    """Custom BERT implementation."""
    def __init__(self, config_dict):
        """Constructs Bert model based on config_dict."""
        super(Bert, self).__init__()
        self.config = Config.from_dict(config_dict)
        self.embeddings = nn.ModuleDict({
            'token': nn.Embedding(self.config.vocab_size, self.config.hidden_size, padding_idx=0),
            'position': nn.Embedding(self.config.max_position_embeddings, self.config.hidden_size),
            'token_type': nn.Embedding(self.config.type_vocab_size, self.config.hidden_size),
        })

        self.ln = LayerNorm(self.config.hidden_size)
        self.dropout = nn.Dropout(self.config.hidden_dropout_prob)

        self.layers = nn.ModuleList([
            Layer(self.config) for _ in range(self.config.num_hidden_layers)
        ])

        self.pooler = nn.Sequential(OrderedDict([
            ('dense', nn.Linear(self.config.hidden_size, self.config.hidden_size)),
            ('activation', nn.Tanh()),
        ]))

    def save_model(self, path):
        """Save model to a file.

        Args:
            path (str): Path to the file where the model will be saved.
        """
        torch.save(self.state_dict(), path)

    @classmethod
    def load_model(cls, config_dict, path):
        """Load model from a file.

        Args:
            config_dict (dict): Dictionary containing the configuration of the model.
            path (str): Path to the model checkpoint.

        Returns:
            Bert: Bert model loaded from the checkpoint.
        """
        model = cls(config_dict)
        model.load_state_dict(torch.load(path))
        return model

    def forward(self, input_ids, attention_mask=None, token_type_ids=None):
        """Forward pass of the model to generate embeddings.

        Args:
            input_ids (torch.Tensor): Input tensor of shape [batch_size, seq_len] containing token ids of tokens in the input sequence.
            attention_mask (torch.Tensor, optional): Mask to avoid performing attention on padding token indices. Defaults to None.
            token_type_ids (torch.Tensor, optional): Segment token indices to indicate first and second portions of the inputs. Defaults to None.

        Returns:
            Tuple[torch.Tensor, torch.Tensor]: Tuple of output embeddings of shape [batch_size, seq_len, hidden_size] and pooled output of shape [batch_size, hidden_size].
        """
        position_ids = torch.arange(input_ids.size(1), dtype=torch.long, device=input_ids.device)
        position_ids = position_ids.unsqueeze(0).expand_as(input_ids)
        if attention_mask is None:
            attention_mask = torch.ones_like(input_ids)
        if token_type_ids is None:
            token_type_ids = torch.zeros_like(input_ids)

        # BUG 7: Embeddings should be summed together, not concatenated
        x = (self.embeddings['token'](input_ids) +
             self.embeddings['position'](position_ids) +
             self.embeddings['token_type'](token_type_ids))
        x = self.dropout(self.ln(x))

        for layer in self.layers:
            x = layer(x, attention_mask)

        o = self.pooler(x[:, 0])
        return x, o


In [ ]:
from transformers import BertTokenizer

config_dict = {
    'vocab_size': 30000,  # Vocabulary size of the pre-trained BERT model
    'hidden_size': 128,  # Size of the hidden layers
    'num_hidden_layers': 2,  # Number of transformer blocks or layers
    'num_attention_heads': 2,  # Number of attention heads in the multi-head attention layers
    'intermediate_size': 512,  # Size of the intermediate (feed-forward) layers
    'dropout_prob': 0.1,  # Dropout probability for regularization
    'max_position_embeddings': 512,  # Maximum position index for positional embeddings
    'type_vocab_size': 2,  # Number of segments (e.g., sentence A and sentence B for NSP task)
    'initializer_range': 0.02,  # Range for weight initialization
}
# Create and save the model
model = Bert(config_dict)
model.save_model("/content/my_custom_bert_model.pth")

# Load the model for predictions
loaded_model = Bert.load_model(config_dict, "/content/my_custom_bert_model.pth")

# Perform predictions
input_ids = torch.randint(0, 10000, (32, 128))  # Example input
attention_mask = torch.ones_like(input_ids)  # Assuming no padding in the example

# Get output embeddings and pooled output
output_embeddings, pooled_output = loaded_model(input_ids, attention_mask)

# Now, let's use a tokenizer for processing a sentence
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


In [ ]:
from datasets import load_dataset

dataset = load_dataset("setfit/20_newsgroups")

# Choose a sentence from the dataset for prediction
sentence = dataset["train"][18]["text"]

# Tokenize the sentence
tokens = tokenizer(sentence, return_tensors='pt', padding=True, truncation=True, max_length=512)

# Perform prediction
with torch.no_grad():
    output_embeddings, pooled_output = model(**tokens)

# Assuming you have a linear layer for classification, apply softmax to get probabilities
linear_layer = torch.nn.Linear(config_dict['hidden_size'], 20)  # Replace num_classes with your actual number of classes
logits = linear_layer(pooled_output)
probabilities = torch.nn.functional.softmax(logits, dim=-1)

# Print the predicted probabilities
print("Predicted Probabilities:", probabilities.tolist())


/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Predicted Probabilities: [[0.0498465821146965, 0.057605985552072525, 0.03751376271247864, 0.06362807750701904, 0.03204759210348129, 0.05146433040499687, 0.056139227002859116, 0.05091714859008789, 0.035728730261325836, 0.06136772036552429, 0.038834139704704285, 0.07049059867858887, 0.0482066385447979, 0.038755446672439575, 0.04320911318063736, 0.04162205010652542, 0.041218679398298264, 0.08356966823339462, 0.050484444946050644, 0.04735006392002106]]


In [ ]:
# Access the 19th row in the "text" column
text_19th_row = dataset['train']['text'][18]

print(text_19th_row)

I would like to be able to amplify a voltage signal which is
output from a thermocouple, preferably by a factor of
100 or 1000 ---- so that the resulting voltage can be fed
more easily into a personal-computer-based ADC data
acquisition card.

Might anyone be able to point me to references to such
circuits?  I have seen simple amplifier circuits before, but
I am not sure how well they work in practice.

In this case, I'd like something which will amplify sufficiently
"nicely" to be used for thermocouples (say, a few degrees
accuracy or better).


In [ ]:
classes = dataset["train"][18]["label"]

print(classes)

12


In [ ]:
# Example sentence
sentence = "This is an example sentence."

# Tokenize the input sentence
tokens = tokenizer(sentence, return_tensors='pt')
input_ids = tokens['input_ids']
attention_mask = tokens['attention_mask']

# Perform predictions on the sentence
output_embeddings, pooled_output = loaded_model(input_ids, attention_mask)

# Output the results
print("Output Embeddings Shape:", output_embeddings.shape)
print("Pooled Output Shape:", pooled_output.shape)

Output Embeddings Shape: torch.Size([1, 8, 128])
Pooled Output Shape: torch.Size([1, 128])


In [ ]:
print(output_embeddings)

tensor([[[-0.0135,  1.7678, -2.6319,  ..., -0.4088,  1.0544, -0.0797],
         [-1.2703, -1.2979, -0.5269,  ..., -0.4693,  1.0385,  1.6049],
         [ 0.9008,  0.6688, -2.7968,  ...,  1.5679, -0.9866,  0.3960],
         ...,
         [-0.1527, -0.2476, -0.4779,  ...,  0.6273,  0.5653, -0.5120],
         [ 2.0031,  1.5730, -1.4441,  ..., -0.0067, -0.7860,  0.7499],
         [ 0.7912, -0.2839, -2.2449,  ...,  1.2883, -0.2629, -0.3554]]],
       grad_fn=<AddBackward0>)
